In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
np.random.seed(42)

In [ ]:
#Initial preprocessing
#Separates the data for Oslo airport, executed for each yearly csv
df = df[df['IATA'].str.startswith('OSL')] 
df.to_csv('Avinor_AirTraffic2019_OSL.csv') 

In [ ]:
#Initial passenger data preprocessing
#Execued for each yearly csv
#Creates a new column in the df 'originaldestionation'
#This new column is a combination of DepartureArrival (AvgangAnkomst) and InnlandUtland (DomesticInternational)
#Data is grouped and passenger amount (termnalpassasjerer) is summed for each hourly index 
df = pd.read_csv('Avinor_AirTraffic2019_OSL.csv',encoding='latin-1')

df['OriginDestination'] = df['AvgangAnkomst'] + '-' + df['InnlandUtland']
df['KortDato'] = pd.to_datetime(df['KortDato'])

hourly_passengers = df.groupby(['KortDato', 'Time', 'OriginDestination'])['Termnalpassasjerer'].sum().reset_index()
hourly_passengers = hourly_passengers.groupby(['KortDato', 'Time'])['Termnalpassasjerer'].sum().reset_index()
hourly_passengers = hourly_passengers.set_index(['KortDato', 'Time']).unstack('Time').stack('Time', dropna=False).reset_index()
hourly_passengers = hourly_passengers.fillna(0)

#Export only date hour and passenger amount
hourly_passengers = hourly_passengers[['KortDato', 'Time', 'Termnalpassasjerer']]
hourly_passengers.to_csv('Avinor_AirTraffic2019_OSL_FINAL.csv', index=False)

In [ ]:
#Weather data preprocessing. Since the data is irregular, the data is processed using only the index nearest to a round hour
#Air temperature and Mean wind speed are the two meteorological variables
#Executed once for each yearly csv
df_being_edited = '2009WeatherData.csv'
df = pd.read_csv(df_being_edited)

df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d.%m.%Y %H:%M')
df.set_index('Datetime', inplace=True)
df_hourly = df.resample('H').nearest()

cols = df_hourly[['Air temperature', 'Mean wind speed']]
cols.to_csv(df_being_edited)

In [ ]:
#Finds missing values in passenger data, dates and file name changed for each year
#Also prints the amount of missing values found
#Missing value rows are removed, with the corresponding rows in weather data also removed 
#Executed once for each yearly csv
start_date = datetime(2019, 1, 1) 
end_date = datetime(2019, 12, 31, 23) 
all_datetimes = [start_date + timedelta(hours=i) for i in range((end_date - start_date).days * 24 + 24)]

data = []
with open('Avinor_AirTraffic2019_OSL_FINAL.csv', 'r') as f: 
    next(f) 
    for line in f:
        date_str, hour_str, passengers = line.strip().split(',')
        date = datetime.strptime(date_str, '%d/%m/%Y')
        hour = int(hour_str)
        data.append((date, hour))

missing_datetimes = set(all_datetimes) - set(datetime.combine(date, datetime.min.time()) + timedelta(hours=hour) for date, hour in data)
x = 0
for dt in sorted(missing_datetimes):
    x += 1
    print(dt.strftime('%d/%m/%Y %H'))
print(x)
x = 0

In [ ]:
#Finds missing dates in weather CSVs, dates and file name changed for each year
#Also prints the amount of missing values found
#Missing value rows are removed, with the corresponding rows in passenger data also removed 
#Executed once for each yearly csv
start_date = datetime(2011, 1, 1) 
end_date = datetime(2011, 12, 31, 23) 
all_datetimes = [start_date + timedelta(hours=i) for i in range((end_date - start_date).days * 24 + 24)]

data = []
with open('2011WeatherData_Final.csv', 'r') as f: 
    next(f) 
    for line in f:
        datetime_str = line.strip().split(',')[0]
        datetime_obj = datetime.strptime(datetime_str, '%d/%m/%Y %H:%M')
        data.append(datetime_obj)

missing_datetimes = set(all_datetimes) - set(data)

y = 0
for dt in sorted(missing_datetimes):
    y += 1
    print(dt.strftime('%d/%m/%Y %H:%M:%S'))
print(y)

In [ ]:
#The data is now ready to be merged, which is done using excel
#This means that the weather data of air temperature and mean wind speed is merged chronologically with the passenger csv
#The resulting yearly csv use the datetime, hourly and passenger amount features from the passenger data
#Aswell as the air temperature and mean wind speed from the weather data
#Preprocessing after this point and onward is done on these new merged yearly csvs

In [ ]:
#Dataset being edited
dataset_name = 'Air2019_Final.csv'
data = pd.read_csv(dataset_name)

In [ ]:
#The datetime feature 'KortDato' is converted to a pandas datetime object
#The day month and yearly components of the datetime are extracted as their own standalone features
#Data is then just reindexed, with passengers column renamed for clarity
data['KortDato'] = pd.to_datetime(data['KortDato'], format='%d/%m/%Y')
data['Day'] = data['KortDato'].dt.day
data['Month'] = data['KortDato'].dt.month
data['Year'] = data['KortDato'].dt.year
data = data.reindex(columns=['Time','Day','Month','Year','Termnalpassasjerer','Air temperature','Mean wind speed'])
data = data.rename(columns={'Termnalpassasjerer':'Passengers'})

In [ ]:
#inspect
data

In [ ]:
#The following preprocessing is executed once for each yearly CSV
#Checks for null, missing values, fixes wrong data types to numeric for weather variables
#Confirms everything is in order and saves the csv
#Check for null values
mask = data[['Year','Passengers','Air temperature','Mean wind speed']].isnull()
missing_data = data[mask.any(axis=1)]
missing_data

#check datatype for columns
result = data.dtypes
print(result)

#Inspect unique values
print('Air Temperature:')
print(data['Air temperature'].unique())
print('Mean wind speed:')
print(data['Mean wind speed'].unique())

#convert wrong datatypes to numeric
data['Air temperature'] = pd.to_numeric(data['Air temperature'], errors='coerce')
data['Mean wind speed'] = pd.to_numeric(data['Mean wind speed'], errors='coerce')

print('Air Temperature:')
print(data['Air temperature'].unique())
print('Mean wind speed:')
print(data['Mean wind speed'].unique())

#drop null and save
data = data.dropna()
data.to_csv(dataset_name, index=False)

In [ ]:
#Only when all the previous preprocessing is done for all individual yearly csv
#Can then the files be merged into the complete csv

In [ ]:
#Merges all the finished yearly csvs, chronologically, into a complete csv
files_to_merge = ['Air2009_Final.csv', 'Air2010_Final.csv', 'Air2011_Final.csv',
                  'Air2012_Final.csv', 'Air2013_Final.csv', 'Air2014_Final.csv',
                  'Air2015_Final.csv', 'Air2016_Final.csv', 'Air2017_Final.csv', 
                  'Air2018_Final.csv', 'Air2019_Final.csv']
data_frames = []

for file in files_to_merge:
    df = pd.read_csv(file, index_col=None, header=0)
    data_frames.append(df)

merged_df = pd.concat(data_frames, axis=0, ignore_index=True)

merged_df.to_csv('Air2009-2019_Complete.csv', index=False)

In [ ]:
#Inspecting the complete csv
merged_df = pd.read_csv('Air2009-2019_Complete.csv', index_col=None, header=0)

print('Number of rows:', len(merged_df))
print('Data types of columns:')
print(merged_df.dtypes)
print(merged_df.head())
print(merged_df.tail())

In [ ]:
#Inspecting random samples 
#print x amount of samples of 10 consecutive rows
sample_size = 10
max_index = len(merged_df) - sample_size

for i in range(20):
    start_index = np.random.randint(0, max_index)
    
    end_index = start_index + sample_size
    sample = merged_df.iloc[start_index:end_index]
    print(f'Sample {i+1}:')
    print(sample)
    print()